**This code initializes an EEG data recording system.**

- It sets up a scanner to detect BrainBit sensors.
- Once sensors are found, it connects to the first available one and processes resistance data.
- A global `data_list` stores EEG signal data recorded via the sensor.
- Functions `start_recording` and `stop_recording` handle the recording process, while `save_to_csv` saves the data to a CSV file.
- `cleanup` ensures proper disconnection and resource management for sensors and the scanner.

In [ ]:
import time
import threading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from neurosdk.scanner import Scanner
from neurosdk.brainbit_sensor import BrainBitSensor
from neurosdk.cmn_types import *

# Global variables
devices = []
recording = False
data_list = []



# Set up scanner
scanner = Scanner([SensorFamily.LEBrainBitFlex])

def sensor_found(scanner, sensors):
    """
    Identifies and processes found sensors from a given scanner, then updates
    global device information with the provided sensor list. This function
    iterates through the sensors, printing their respective name and address.

    :param scanner: Scanner object responsible for detecting sensors
    :param sensors: List of sensors detected by the scanner
    :type sensors: list
    :return: None
    """
    global devices
    devices = sensors
    for i, sensor in enumerate(sensors):
        print(f"Sensor {i+1}: {sensor.Name}, Address: {sensor.Address}")

scanner.sensorsChanged = sensor_found
scanner.start()
time.sleep(5)  # Allow time for device discovery
scanner.stop()



# Connect to first available device
if devices:
    sensor_info = devices[0]
    sensor = scanner.create_sensor(sensor_info)
    print("Connected to:", sensor.name)
else:
    print("No devices found")
    exit()

# Check resistance
def on_resistance_data(sensor, data):
    """
    Processes resistance data reported by a sensor.

    This function is used to handle resistance data provided by a specific sensor.
    The resistance value might be utilized for further processing, logging, or
    monitoring sensor conditions.

    :param sensor: An object representing the sensor that reports the data.
                   It can be used to identify or interact with the sensor origin.
    :type sensor: Any

    :param data: The resistance data collected by the sensor.
                 It generally represents sensor readings that might vary over
                 time or based on environmental factors.
    :type data: Any

    :return: None
    :rtype: None
    """
    print("Resistance Data:", data)

sensor.resistDataReceived = on_resistance_data
sensor.exec_command(SensorCommand.StartResist)
time.sleep(5)
sensor.exec_command(SensorCommand.StopResist)


def on_signal_data_received(sensor, data):
    """
    Handles the signal received from a sensor, processes the data packages, and appends
    timestamped samples to a global data list if recording is enabled. If the data packages
    lack the required 'Samples' attribute, it generates a warning message.

    :param sensor:
        The sensor object that sends the signal.
    :param data: list
        A list of data packages transmitted by the sensor. Each package is expected to
        have a 'Samples' attribute containing the sample data.
    :return: None
    """
    global data_list, recording
    if recording:
        timestamp = time.time()
        for package in data:  # Iterate through each package in the data list
            if hasattr(package, "Samples"):  # Ensure the package has a Samples attribute
                data_list.append([timestamp] + package.Samples)
            else:
                print(f"Warning: Package {package} does not contain the attribute 'Samples'")

sensor.signalDataReceived = on_signal_data_received

def start_recording():
    """
    Starts the recording process by initializing the necessary structures and
    executing the start signal command for the sensor.

    The function initializes an empty list to store sensor data and sets the
    recording status to True. It also sends a command to the connected sensor
    to begin recording signals.

    :return: None
    """
    global recording, data_list
    data_list = []
    recording = True
    sensor.exec_command(SensorCommand.StartSignal)
    print("Recording started...")

def stop_recording():
    """
    Stops the ongoing recording process.

    This function sets the global variable `recording` to False, signaling that
    the recording process should stop. It also sends a stop signal command
    to the sensor device using the `SensorCommand.StopSignal` parameter.
    Finally, it prints a confirmation message to indicate that the recording
    has been stopped.

    :raises NameError: If global `recording` or `sensor` is not initialized.
    :return: None
    """
    global recording
    recording = False
    sensor.exec_command(SensorCommand.StopSignal)
    print("Recording stopped...")

def save_to_csv(filename="eeg_data.csv"):
    """
    Saves the recorded EEG data to a CSV file. The function creates a DataFrame
    from the `data_list`, which is expected to hold timestamped data for multiple
    channels. The data is then written to a CSV file specified by the `filename`
    parameter.

    :param filename: The name of the CSV file where the EEG data will be saved.
        If not specified, defaults to "eeg_data.csv".
    :return: None
    """
    if data_list:
        df = pd.DataFrame(data_list, columns=["Timestamp"] + [f"Channel_{i}" for i in range(len(data_list[0]) - 1)])
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No data recorded")

def cleanup():
    """
    Cleans up resources used by the system such as sensor and scanner.

    The function ensures proper disconnection and cleanup of global objects
    'sensor' and 'scanner' to release resources and prevent potential memory
    leaks or resource contention. If the objects are active, it logs the
    respective disconnection or release action before clearing their references.

    :raises AttributeError: If the function tries to access attributes
       of the objects being cleaned up without ensuring they exist.
    :raises AnyErrorType: Specify additional error types raised by the
       actual cleanup logic specific to 'scanner' or 'sensor'.

    :return: None
    """
    global sensor, scanner

    if sensor:
        print("Disconnecting sensor...")

        sensor = None  # Clear the reference
    if scanner:
        print("Releasing scanner...")
        # Add actual scanner cleanup logic here
        scanner = None  # Clear the reference


**This function `eeg_state` records EEG data for a specified duration (default 10 seconds), saves it to a CSV file, and cleans up resources. It starts recording, waits for the specified duration, stops recording, saves the data to a CSV, and returns the recorded data as a list.**

#TODO Implementation of a ML mechanism, therefor start and stop is seperated

#TODO Threading


In [2]:
def eeg_state(duration=10, filename="eeg_data.csv"):
    """
    Records EEG data for a specified duration, saves it to a given filename,
    and clears up resources post-recording. The function returns the
    recorded EEG data as a list. This supports recording EEG signals for
    session analyses or other purposes.

    :param duration: Duration (in seconds) for which EEG data needs
        to be recorded. Default is 10.
    :param filename: Name of the CSV file where the recorded EEG data
        will be saved. Default is "eeg_data.csv".
    :return: A list containing the recorded EEG data.
    """
    global data_list
    data_list = []
    start_recording()
    time.sleep(duration)
    stop_recording()
    save_to_csv(filename=filename)
    cleanup()
    return data_list

In [ ]:
import threading
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd

# Global variables for threading and real-time data visualization
streaming = False
live_data = []

# Function to handle real-time data updates
def start_streaming():
    """
    Starts streaming EEG data in a separate thread. Updates the global
    variable `live_data` with streamed data for visualization.
    """
    global streaming, live_data, data_list
    streaming = True
    data_list = []
    live_data = []

    def stream_data():
        start_recording()
        print("Streaming started...")
        while streaming:
            if data_list:
                # Append a copy of the latest data from `data_list`
                live_data.extend(data_list)
                # Clear `data_list` periodically to avoid redundant data processing
                data_list.clear()
            time.sleep(0.1)  # Small delay to simulate real-time streaming
        stop_recording()

    # Start the streaming thread
    threading.Thread(target=stream_data, daemon=True).start()

def stop_streaming():
    """
    Stops the EEG data streaming, halting the background thread.
    """
    global streaming
    streaming = False
    print("Streaming stopped.")

# Function to update the live EEG graph
def update_eeg_plot(frame):
    """
    Updates the real-time EEG plot with new data.

    :param frame: Animation frame parameter (used internally by FuncAnimation).
    """
    global live_data
    if live_data:
        # Assuming the data format: [Timestamp, Channel_1, Channel_2, ...]
        df = pd.DataFrame(live_data, columns=["Timestamp"] + [f"Channel_{i}" for i in range(len(live_data[0]) - 1)])
        for i, line in enumerate(lines):
            line.set_data(df["Timestamp"], df[f"Channel_{i}"])
        ax.relim()
        ax.autoscale_view()

# Initialize the plot for real-time EEG graph
fig, ax = plt.subplots(figsize=(12, 6))
lines = []
for i in range(4):  # Assuming 4 EEG channels (adjust based on data available)
    line, = ax.plot([], [], label=f'Channel {i}')
    lines.append(line)

ax.set_title("Real-Time EEG Data Visualization")
ax.set_xlabel("Timestamp")
ax.set_ylabel("Signal Value")
ax.legend()
ax.grid(True)

# Animation for real-time graph updates
ani = FuncAnimation(fig, update_eeg_plot, interval=100)

# Starting the streaming
start_streaming()

# Show the plot
plt.show()

# Stop streaming once the plot is closed
stop_streaming()
cleanup()

In [3]:
eeg_state(600, "eeg_data_10m.csv")


Recording started...
Recording stopped...
Data saved to eeg_data_10m.csv
Disconnecting sensor...
Releasing scanner...


[[1741010401.512202,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953],
 [1741010401.512202,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953],
 [1741010401.512202,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953],
 [1741010401.512202,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953],
 [1741010401.5420392,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.4000000635782953,
  -0.40000006357

In [ ]:
import matplotlib.pyplot as plt

# Plot EEG data for each channel
plt.figure(figsize=(12, 6))
for i in range(1, len(eeg_data.columns)/10):  # Exclude Timestamp and Cluster columns
    plt.plot(eeg_data['Timestamp'], eeg_data[f'Channel_{i-1}'], label=f'Channel {i-1}')

plt.title("EEG Data Visualization")
plt.xlabel("Timestamp")
plt.ylabel("Signal Value")
plt.legend()
plt.grid(True)
plt.show()